## Fetch GDELT Data

In [1]:
import requests
import lxml.html as lh
import os
import zipfile
import glob
import operator
import pandas as pd

### 1. Identify 2017 file links from the data source website

In [2]:
year_to_analyze = '2017'
# fetch all file names from source website
gdelt_data_links = 'http://data.gdeltproject.org/events/'

response = requests.get(gdelt_data_links + 'index.html')
html_content = lh.fromstring(response.content)
all_links = html_content.xpath("//*/ul/li/a/@href")

# identify 2017 files
file_list = [x for x in all_links if x[0:4]==year_to_analyze]
print('Total 2017 files identified:', len(file_list))
print('Sample of file names:', file_list[:3])

Total 2017 files identified: 107
Sample of file names: ['20170417.export.CSV.zip', '20170416.export.CSV.zip', '20170415.export.CSV.zip']


### 2. Download Data using the links obtained

In [3]:
pwd = os.getcwd()
data_dir = pwd + '/../data/'
us_code = 'US'
countries_interested_codes = set(['UK', 'CA', 'CH', 'MX', 'IN'])


In [4]:
for file_name in file_list:
    file_to_save = data_dir + 'compressed/' + file_name
    # Avoid downloading if already exists
    if not os.path.isfile(file_to_save): 
        response = requests.get(gdelt_data_links + file_name)
        with open(file_to_save, 'wb') as f:
            f.write(response.content) 
print('Compressed zip files are saved at', data_dir + 'compressed/')            